# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# input csv
input_csv = "../output_data/cities.csv"

#import pretty print
import pprint as pprint

#for sleep
import time

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather = pd.read_csv("../output_data/cities.csv",index_col=0)
city_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ushuaia,-54.8000,-68.3000,46.40,57.0,75.0,33.38,AR,1.615672e+09
1,saskylakh,71.9167,114.0833,-10.23,98.0,42.0,4.72,RU,1.615672e+09
2,baft,29.2340,56.6057,47.35,66.0,0.0,2.21,IR,1.615672e+09
3,eyl,7.9803,49.8164,77.56,86.0,81.0,12.17,SO,1.615672e+09
4,bouar,5.9340,15.5960,70.43,73.0,97.0,4.03,CF,1.615672e+09
...,...,...,...,...,...,...,...,...,...
551,baia de aries,46.3833,23.2833,34.00,93.0,1.0,10.36,RO,1.615672e+09
552,honiara,-9.4333,159.9500,77.43,86.0,100.0,2.10,SB,1.615672e+09
553,mount gambier,-37.8333,140.7667,41.00,93.0,40.0,5.75,AU,1.615672e+09
554,dumas,35.8656,-101.9732,64.00,100.0,40.0,10.36,US,1.615672e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# !jupyter nbextension enable --py gmaps

In [13]:
# Import API key
from api_keys import g_key
if g_key == "YOUR KEY HERE!":
    import sys
    path = "/Users/matth/Desktop"
    sys.path.insert(0, path)
    from API_Keys import api_key
    g_key= api_key["Google Maps Platform"]
# configure gmaps with api key
gmaps.configure(api_key=g_key)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(city_weather[["Lat","Lng"]], weights=city_weather["Humidity"], 
                                 dissipating=False, max_intensity=city_weather["Humidity"].max(),
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)
g_key=[]
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# * Narrow down the DataFrame to find your ideal weather condition. For example:
#   * A max temperature lower than 80 degrees but higher than 70.
#   * Wind speed less than 10 mph.
#   * Zero cloudiness.
#   * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
ideal_weather = city_weather[(city_weather["Max Temp"]>70)&\
              (city_weather["Max Temp"]<80)&\
              (city_weather["Wind Speed"]<10)&\
              (city_weather["Cloudiness"]==0)].reset_index(drop=True)
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,saint-philippe,-21.3585,55.7679,75.20,78.0,0.0,4.61,RE,1.615672e+09
1,salalah,17.0151,54.0924,75.20,73.0,0.0,4.61,OM,1.615672e+09
2,bilma,18.6853,12.9164,79.48,11.0,0.0,5.30,NE,1.615672e+09
3,surin,14.8818,103.4936,79.00,66.0,0.0,8.61,TH,1.615672e+09
4,kabin buri,13.9511,101.7177,79.23,78.0,0.0,3.20,TH,1.615672e+09
5,yazman,29.1212,71.7446,70.03,32.0,0.0,7.58,PK,1.615672e+09
6,caravelas,-17.7125,-39.2481,76.08,80.0,0.0,5.97,BR,1.615672e+09
7,saint-joseph,-21.3667,55.6167,75.20,78.0,0.0,4.61,RE,1.615672e+09
8,coihaique,-45.5752,-72.0662,73.40,33.0,0.0,5.75,CL,1.615672e+09
9,san vicente,-26.6167,-54.1333,75.38,45.0,0.0,4.70,AR,1.615672e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather
hotel_df["Hotel Name"]=""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,saint-philippe,-21.3585,55.7679,75.20,78.0,0.0,4.61,RE,1.615672e+09,
1,salalah,17.0151,54.0924,75.20,73.0,0.0,4.61,OM,1.615672e+09,
2,bilma,18.6853,12.9164,79.48,11.0,0.0,5.30,NE,1.615672e+09,
3,surin,14.8818,103.4936,79.00,66.0,0.0,8.61,TH,1.615672e+09,
4,kabin buri,13.9511,101.7177,79.23,78.0,0.0,3.20,TH,1.615672e+09,
5,yazman,29.1212,71.7446,70.03,32.0,0.0,7.58,PK,1.615672e+09,
6,caravelas,-17.7125,-39.2481,76.08,80.0,0.0,5.97,BR,1.615672e+09,
7,saint-joseph,-21.3667,55.6167,75.20,78.0,0.0,4.61,RE,1.615672e+09,
8,coihaique,-45.5752,-72.0662,73.40,33.0,0.0,5.75,CL,1.615672e+09,
9,san vicente,-26.6167,-54.1333,75.38,45.0,0.0,4.70,AR,1.615672e+09,


In [7]:
# Import API key
from api_keys import g_key
if g_key == "YOUR KEY HERE!":
    import sys
    path = "/Users/matth/Desktop"
    sys.path.insert(0, path)
    from API_Keys import api_key
    g_key= api_key["Google Maps Platform"]
# configure gmaps with api key
gmaps.configure(api_key=g_key)

# https://developers.google.com/maps/documentation/places/web-service/supported_types?hl=en_US
#supported types for hotels is probably "lodging"
#URL example found at https://developers.google.com/maps/documentation/places/web-service/get-api-key?hl=en_US
#https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=500&types=food&name=harbour&key=YOUR_API_KEY
radius=5000
types="lodging"
key=g_key

for i in hotel_df.index:
    location=f"{hotel_df['Lat'][i]},{hotel_df['Lng'][i]}"
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&types={type}&key={g_key}"
    try:
        response=requests.get(url).json()
        time.sleep(0.5)
        hotel_df["Hotel Name"][i]=response["results"][1]["name"]
    except:
        hotel_df["Hotel Name"][i]=float("NaN")
#clears g_key (api key) after use.
g_key=[]
url = []

<ipython-input-7-86d49dd2c6e9>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"][i]=response["results"][1]["name"]


In [8]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,saint-philippe,-21.3585,55.7679,75.20,78.0,0.0,4.61,RE,1.615672e+09,"Chambres d'hôte ""La Trinité"""
1,salalah,17.0151,54.0924,75.20,73.0,0.0,4.61,OM,1.615672e+09,HAMDAN PLAZA HOTEL SALALAH
2,bilma,18.6853,12.9164,79.48,11.0,0.0,5.30,NE,1.615672e+09,Централна джамия Билма
3,surin,14.8818,103.4936,79.00,66.0,0.0,8.61,TH,1.615672e+09,One Fu Hotel
4,kabin buri,13.9511,101.7177,79.23,78.0,0.0,3.20,TH,1.615672e+09,Kabin Buri Central Mosque
5,yazman,29.1212,71.7446,70.03,32.0,0.0,7.58,PK,1.615672e+09,Zahid Clinic
6,caravelas,-17.7125,-39.2481,76.08,80.0,0.0,5.97,BR,1.615672e+09,Conselho Tutelar
7,saint-joseph,-21.3667,55.6167,75.20,78.0,0.0,4.61,RE,1.615672e+09,"""Plantation Bed and Breakfast"
8,coihaique,-45.5752,-72.0662,73.40,33.0,0.0,5.75,CL,1.615672e+09,Diego De Almagro Coyhaique
9,san vicente,-26.6167,-54.1333,75.38,45.0,0.0,4.70,AR,1.615672e+09,Banco Macro Sucursal 13 San Pedro


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, \
                                 info_box_content=[f"{info}" for info in hotel_info])

# Add layers and Display figure
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)


fig

Figure(layout=FigureLayout(height='420px'))